# Sentiment Analysis in the Browser

In this notebook, we will show how to create a `.air` file to perform sentiment analysis in the browser using a neural network.

## Dependencies

For this notebook, the following dependencies are required:

- `scikit-learn`
- `tensorflow`
- `aisquared`

All of these are available on [pypi](https://pypi.org) via `pip`.  The following cell also runs the commands to install these dependencies as well as imports them into the notebook environment, along with TensorFlow (which is a dependency of the `mann` package).

In [1]:
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import aisquared

/Users/jwrenn4/miniforge3/lib/python3.9/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


## Model Creation

Now that the required packages have been installed and imported, it is time to create the sentiment analysis model.  To do this, we have to first download and preprocess the data, train the model, and then package the model in the `.air` format.  The following cells will go through an in-depth explanation of each of the steps in this process.

In [2]:
# Loading the data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(
    num_words = 10000,
    skip_top = 0,
    start_char = 1,
    oov_char = 2,
    index_from = 3
)
x_train = tf.keras.preprocessing.sequence.pad_sequences(
    x_train,
    maxlen = 512,
    padding = 'post',
    truncating = 'post'
)
x_test = tf.keras.preprocessing.sequence.pad_sequences(
    x_test,
    maxlen = 512,
    padding = 'post',
    truncating = 'post'
)

# Get the vocabulary
vocab = tf.keras.datasets.imdb.get_word_index()

# Add 2 to each vocab value to ensure matching with the needed values
vocab = {
    k : v + 2 for k, v in vocab.items()
}


In [3]:
# Create the model

input_layer = tf.keras.layers.Input(x_train.shape[1:])
embedding_layer = tf.keras.layers.Embedding(
    10000,
    4
)(input_layer)
x = tf.keras.layers.Flatten()(embedding_layer)
for _ in range(5):
    x = tf.keras.layers.Dense(256, activation = 'relu')(x)
    x = tf.keras.layers.Dropout(0.1)(x)
output_layer = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model = tf.keras.models.Model(input_layer, output_layer)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Metal device set to: Apple M1
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512)]             0         
                                                                 
 embedding (Embedding)       (None, 512, 4)            40000     
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (N

In [4]:
# Train the model
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(
    x_train,
    y_train.reshape(-1,1),
    epochs = 20,
    batch_size = 512,
    validation_split = 0.2,
    callbacks = tf.keras.callbacks.EarlyStopping(min_delta = 0.001, patience = 3, restore_best_weights = True)
)

Epoch 1/20


2022-09-13 08:18:55.282574: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


40/40 [==============================] - 29s 706ms/step - loss: 0.6921 - accuracy: 0.5110 - val_loss: 0.6856 - val_accuracy: 0.5420
Epoch 2/20
40/40 [==============================] - 28s 691ms/step - loss: 0.5290 - accuracy: 0.7290 - val_loss: 0.4790 - val_accuracy: 0.8010
Epoch 3/20
40/40 [==============================] - 25s 623ms/step - loss: 0.2701 - accuracy: 0.8917 - val_loss: 0.3707 - val_accuracy: 0.8472
Epoch 4/20
40/40 [==============================] - 27s 666ms/step - loss: 0.1445 - accuracy: 0.9506 - val_loss: 0.4777 - val_accuracy: 0.8452
Epoch 5/20
40/40 [==============================] - 25s 620ms/step - loss: 0.0700 - accuracy: 0.9778 - val_loss: 0.7303 - val_accuracy: 0.8324
Epoch 6/20
40/40 [==============================] - 23s 591ms/step - loss: 0.0474 - accuracy: 0.9849 - val_loss: 0.6667 - val_accuracy: 0.8454


In [5]:
# Check model performance
preds = (model.predict(x_test) >= 0.5).astype(int)
print('Model Performance on Test Data:')
print('\n')
print(confusion_matrix(y_test, preds))
print(classification_report(y_test, preds))

# Save the model
model.save('SentimentClassifier.h5')

Model Performance on Test Data:


[[10750  1750]
 [ 2239 10261]]
              precision    recall  f1-score   support

           0       0.83      0.86      0.84     12500
           1       0.85      0.82      0.84     12500

    accuracy                           0.84     25000
   macro avg       0.84      0.84      0.84     25000
weighted avg       0.84      0.84      0.84     25000



## Package the Model

Now that the model has been created, we can package the model into a single `.air` file that enables integration into the browser.

To perform this packaging, we will be utilizing the `aisquared` package `DocumentPredictor` class.

In [6]:
harvester = aisquared.config.harvesting.TextHarvester()

preprocessor = aisquared.config.preprocessing.text.TextPreprocessor(
    [
        aisquared.config.preprocessing.text.Tokenize(),
        aisquared.config.preprocessing.text.ConvertToCase(lowercase = True),
        aisquared.config.preprocessing.text.RemoveCharacters(),
        aisquared.config.preprocessing.text.ConvertToVocabulary(vocabulary = vocab, max_vocab = 9999),
        aisquared.config.preprocessing.text.PadSequences(length = 512, pad_location = 'post', truncate_location = 'post')
    ]
)   
analytic = aisquared.config.analytic.LocalModel('SentimentClassifier.h5', 'text')

##we might be missing an analogue to "sequence_length" in the .config package

postprocessor = aisquared.config.postprocessing.BinaryClassification(['positive', 'negative'], 0.5)

renderer = aisquared.config.rendering.DocumentRendering(include_probability = True)

feedback = aisquared.config.feedback.BinaryFeedback(['positive', 'negative'])

aisquared.config.ModelConfiguration(
    'SentimentClassifier',
    harvester,
    preprocessor,
    analytic,
    postprocessor,
    renderer,
    feedback).compile(dtype = 'float16')